This file will be used to contain data processing components

In [28]:
# %pip install boto3 pyspark delta-spark python-dotenv s3fs
# %pip install findspark

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'hadoop-aws'"

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
# You can use the following to set the environment variables in the notebook if you don't set manually access key, secret key and endpoint in minio
os.environ['OBJ_STORAGE_ACCESS_KEY'] = 'PmQYxTR8Ls9xlFUloH1H'
os.environ['OBJ_STORAGE_SECRET_KEY'] = 'XPuYiU74d12m05k5u6BgYHbGibKRTNR1tmdss3yh'
os.environ['OBJ_STORAGE_ENDPOINT'] = 'http://localhost:9000'

In [3]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT')

In [4]:
path_1 = "s3a://data/data-raw/data.json"
path_2 = "s3a://data/data-raw/data2.json"
path_3 = "s3a://data/data-raw/data3.json"

In [5]:
# You need to more configuration if you want to use minio as object storage 
# (hint: maybe you can using .config() method to set the configuration if you want using spark to read/write data from/to minio)


spark = SparkSession.builder \
    .appName("Python Spark SQL with MinIO") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.endpoint", obj_storage_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", obj_storage_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", obj_storage_secret_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .getOrCreate()



In [31]:
#Read json
df_1 = spark.read.option("multiLine", True).json(path_1)
df_2 = spark.read.option("multiLine", True).json(path_2)
df_3 = spark.read.option("multiLine", True).json(path_3)
#Merge json
df = df_1.union(df_2).union(df_3)
# df.show()
# write result.json to minio
df.write.format("json") \
    .mode("overwrite") \
    .json("s3a://data/data-result/result.json")

In [39]:
#read result.json
df = spark.read.json("s3a://data/data-result/result.json")
df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0008| Strawberry Frosted|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5.

In [40]:
#before preprocessing
df.count()

19

In [41]:
#Drop duplicate
df = df.dropDuplicates()
df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0011|  Devil's Food Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5.

In [42]:
#after preprocessing
df.count()

18

In [43]:
#write csv to minio
df.withColumn('batters', col('batters').cast('string')) \
    .withColumn('filling', col('filling').cast('string')) \
    .withColumn('topping', col('topping').cast('string')) \
    .write.csv("s3a://data/data-result/result.csv", header=True, mode="overwrite")

In [44]:
df = spark.read.csv("s3a://data/data-result/result.csv", header=True)
df.show()

+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|             batters|             filling|  id|               name| ppu|             topping| type|
+--------------------+--------------------+----+-------------------+----+--------------------+-----+
|{[{1001, Regular}...|                NULL|0004|              Jelly|0.65|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0011|  Devil's Food Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0010|     Blueberry Cake|0.55|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0007|    Vanilla Frosted|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|[{6001, None}, {6...|0006|     Cinnamon Twist|0.85|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0009|     Chocolate Cake|0.75|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|                NULL|0005|     Custard-Filled|0.75|[{5001, None}, {5.